# Feast - using online and offline stores

Load the credentials

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("../.env-feast")

True

## Offline store (BigQuery)

### Testing get historical features

In [2]:
import pandas as pd
from datetime import datetime
from feast import FeatureStore

# 1. Instantiate the FeatureStore
store = FeatureStore(repo_path="./feast")

# 2. Define your entity dataframe with historical data.
entity_df_sql = """
SELECT 
    userid, 
    movieid, 
    TIMESTAMP_SECONDS(timestamp) AS event_timestamp 
FROM 
    `tesis-master-ciencia-de-datos.feast_staging.ratings` 
TABLESAMPLE SYSTEM (1 PERCENT) 
LIMIT 500000
"""

# 3. Retrieve the historical features using the FeatureService
training_data = store.get_historical_features(
    entity_df=entity_df_sql,
    features=store.get_feature_service("movie_recommender_service_all"),
)

E0000 00:00:1758929485.777040    2973 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [3]:
# 4. Convert the result to a Pandas DataFrame and print it
df = training_data.to_df()

E0000 00:00:1758929519.761773    2973 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [4]:
df

,userid,movieid,event_timestamp,rating,title,genres
0,23346,1,1997-05-23 20:46:08+00:00,3.0,None,None
1,155777,1,2000-12-03 01:25:17+00:00,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,84085,1,1999-11-18 15:23:22+00:00,3.0,None,None
3,111078,1,2005-11-04 20:55:42+00:00,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,123808,1,1997-05-30 13:24:03+00:00,4.0,None,None
...,...,...,...,...,...,...
499995,94770,208679,2019-11-09 23:22:01+00:00,4.0,Deer Boy (2017),Drama|Fantasy
499996,29065,208793,2019-11-14 17:44:06+00:00,3.5,Watchman (2019),Drama|Thriller
499997,115835,208806,2019-11-13 18:06:45+00:00,4.0,First Man on the Moon (2012),Documentary
499998,96399,209063,2019-11-17 20:05:28+00:00,4.0,The Prep School Negro (2012),(no genres listed)


## Online database

### Materialize

In [7]:
import pandas as pd
from datetime import datetime, timedelta
from feast import FeatureStore

# 1. Instantiate the FeatureStore
# Ensure your working directory is the parent of the 'feast' directory
store = FeatureStore(repo_path="./feast")

# --- 2. Define the Overall Materialization Window ---

# The earliest date available in your historical data (based on your 'movies_source' dummy timestamp)
OVERALL_START_DATE = datetime(2003, 1, 5, 0, 0, 0) 
# OVERALL_START_DATE = datetime(1995, 1, 1, 0, 0, 0) 

# The current date, or a chosen cutoff date
OVERALL_END_DATE = datetime.now() 
# OVERALL_END_DATE = datetime(2000, 1, 1, 0, 0, 0)

# Define the period length
YEAR_PERIOD = timedelta(days=100) 

print(f"--- Starting Feature Materialization in 1-Year Chunks ---")
print(f"Overall Data Range: {OVERALL_START_DATE.date()} to {OVERALL_END_DATE.date()}")

# 3. Loop through the time range in 1-year increments
current_start = OVERALL_START_DATE

while current_start < OVERALL_END_DATE:
    
    # Calculate the end date for the current chunk (one year later, or the overall end date)
    current_end = current_start + YEAR_PERIOD
    if current_end > OVERALL_END_DATE:
        current_end = OVERALL_END_DATE

    # Stop the loop if the current chunk is just the remainder
    if current_start >= current_end:
        break
    
    print(f"\nMaterializing chunk: {current_start.date()} to {current_end.date()}")
    
    try:
        # Use store.materialize() to explicitly set both start and end dates
        # Use store.materialize_incremental() if you want Feast to automatically manage
        # the start time based on the last successful run, but for forced chunks,
        # 'materialize' is cleaner. We use 'materialize' here.
        store.materialize(
            feature_views=[
                "user_rating_feature_view",
                "movie_details_feature_view",
            ],
            start_date=current_start,
            end_date=current_end,
        )
        print(f"SUCCESS: Chunk {current_start.date()} completed.")

    except Exception as e:
        print(f"FAILURE: Chunk {current_start.date()} failed.")
        raise e
        # Optionally, break the loop or add retry logic here
        break

    # Advance the start date for the next iteration
    current_start = current_end
    
print("\n--- Overall Materialization Process Finished ---")

# Optional: You can still run the retrieval code here
# training_data = store.get_historical_features(...)
# df = training_data.to_df()

--- Starting Feature Materialization in 1-Year Chunks ---
Overall Data Range: 2003-01-05 to 2025-09-27

Materializing chunk: 2003-01-05 to 2003-04-15
Materializing 2 feature views from 2003-01-05 00:00:00+00:00 to 2003-04-15 00:00:00+00:00 into the redis online store.

user_rating_feature_view:


E0000 00:00:1758935396.005691    3497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


movie_details_feature_view:


E0000 00:00:1758935424.019491    3497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


SUCCESS: Chunk 2003-01-05 completed.

Materializing chunk: 2003-04-15 to 2003-07-24
Materializing 2 feature views from 2003-04-15 00:00:00+00:00 to 2003-07-24 00:00:00+00:00 into the redis online store.

user_rating_feature_view:


...

SUCCESS: Chunk 2025-06-18 completed.

Materializing chunk: 2025-09-26 to 2025-09-27
Materializing 2 feature views from 2025-09-26 00:00:00+00:00 to 2025-09-27 01:09:54+00:00 into the redis online store.

user_rating_feature_view:


E0000 00:00:1758937391.552017    3497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


movie_details_feature_view:


E0000 00:00:1758937394.400458    3497 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


SUCCESS: Chunk 2025-09-26 completed.

--- Overall Materialization Process Finished ---


## Testing

In [3]:
import pandas as pd
from datetime import datetime
from feast import FeatureStore

# 1. Instantiate the FeatureStore
store = FeatureStore(repo_path="./feast")

entity_rows = {
        "userid": [1, 1],
        "movieid": [296, 306]
    }

training_data = store.get_online_features(
    entity_rows=entity_rows,
    features=store.get_feature_service("movie_recommender_service_all"),
)

training_data.to_dict()

AuthenticationError: Authentication required.

In [4]:
store.get_online_features(
    entity_rows={"movieid": list(range(4543, 4555))},
    features=[
        "movie_details_feature_view:genres",
        "movie_details_feature_view:title",
    ],
).to_dict()

{'movieid': [4543,
  4544,
  4545,
  4546,
  4547,
  4548,
  4549,
  4550,
  4551,
  4552,
  4553,
  4554],
 'title': ['Shoot to Kill (1988)',
  'Short Circuit 2 (1988)',
  'Short Circuit (1986)',
  'Vanishing, The (Spoorloos) (1988)',
  'Stormy Monday (1988)',
  'Sunset (1988)',
  'Sweet Hearts Dance (1988)',
  'Switching Channels (1988)',
  'Telephone, The (1988)',
  'Tetsuo, the Ironman (Tetsuo) (1988)',
  'They Live (1988)',
  'To Kill a Priest (1988)'],
 'genres': ['Action|Adventure',
  'Comedy|Sci-Fi',
  'Comedy|Sci-Fi',
  'Drama|Thriller',
  'Crime|Drama',
  'Action|Comedy|Thriller|Western',
  'Drama',
  'Comedy',
  'Comedy',
  'Action|Horror|Sci-Fi|Thriller',
  'Action|Sci-Fi|Thriller',
  'Drama']}